# Sample Melody Generator


In [10]:
import os
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.melody_rnn import melody_rnn_sequence_generator
import tensorflow as tf 

import magenta
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

import pretty_midi
import six
import magenta.music as mm

### Importing MIDI file


In [2]:
twinkle = magenta.music.midi_io.midi_file_to_sequence_proto("./samples/twinkle.mid") 

In [26]:
with tf.gfile.Open("./samples/twinkle.mid", 'rb') as f:
    midi_as_string = f.read()

In [27]:
midi_as_string

b'MThd\x00\x00\x00\x06\x00\x01\x00\x02\x00`MTrk\x00\x00\x00\x13\x00\xffQ\x03\x07\n\xe2\x00\xffX\x04\x04\x02\x18\x08\x00\xff/\x00MTrk\x00\x00\x005\x00\xff\x03\x0creFX Nexus 1\x00\x90<d\x13\x80<d\x1d\x90<d\x13\x80<d\x1d\x90Cd\x13\x80Cd\x1d\x90Cd\x81\x0b\x80Cd\x00\xff/\x00'

In [17]:
midi = pretty_midi.PrettyMIDI(six.BytesIO(midi_as_string))

In [25]:
midi.get_beats(10)

array([], dtype=float64)

In [18]:
help(midi).

Help on PrettyMIDI in module pretty_midi.pretty_midi object:

class PrettyMIDI(builtins.object)
 |  A container for MIDI data in an easily-manipulable format.
 |  
 |  Parameters
 |  ----------
 |  midi_file : str or file
 |      Path or file pointer to a MIDI file.
 |      Default ``None`` which means create an empty class with the supplied
 |      values for resolution and initial tempo.
 |  resolution : int
 |      Resolution of the MIDI data, when no file is provided.
 |  intitial_tempo : float
 |      Initial tempo for the MIDI data, when no file is provided.
 |  
 |  Attributes
 |  ----------
 |  instruments : list
 |      List of :class:`pretty_midi.Instrument` objects.
 |  key_signature_changes : list
 |      List of :class:`pretty_midi.KeySignature` objects.
 |  time_signature_changes : list
 |      List of :class:`pretty_midi.TimeSignature` objects.
 |  lyrics : list
 |      List of :class:`pretty_midi.Lyric` objects.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, mid

### Bundles

In [16]:
melody = ["basic_rnn", "lookback_rnn", "attention_rnn"]
performance = [
    "performance",
    "performance_with_dynamics",
    "density_conditioned_performance_with_dynamics",
    "pitch_conditioned_performance_with_dynamics",
    "multiconditioned_performance_with_dynamics",
    "optional_multiconditioned_performance_with_dynamics",
]
our = [
    "classical_attention_rnn"
]

# Constants.
BUNDLE_DIR = r'./bundles'
MODEL_NAME = melody[2]
BUNDLE_NAME = MODEL_NAME + '.mag'

# Download bundle .mag file 
# mm.notebook_utils.download_bundle(BUNDLE_NAME, BUNDLE_DIR)

In [22]:
bundle = mm.sequence_generator_bundle.read_bundle_file(os.path.join(BUNDLE_DIR, BUNDLE_NAME))

# Performance RNN
generator_per = performance_sequence_generator.get_generator_map()

# Melody RNN
generator_mel = melody_rnn_sequence_generator.get_generator_map()

generator = generator_mel[MODEL_NAME](checkpoint=None, bundle=bundle)
generator.initialize()


generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1  # Higher is more random; 1.0 is default. 
generate_section = generator_options.generate_sections.add(start_time=0, end_time=30)


INFO:tensorflow:Restoring parameters from C:\Users\georg\AppData\Local\Temp\tmp33g6zyz1\model.ckpt


In [20]:
# Generate from scratch
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -165.926132 


In [19]:
# Generate from file
sequence = generator.generate(file, generator_options)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -166.956604 


In [21]:
# Play and view this masterpiece.
mm.plot_sequence(sequence)
mm.play_sequence(sequence)

Loading BokehJS ...